In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-04-05 16:01:55--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2024-04-05 16:01:55 (21.6 MB/s) - ‘input.txt.2’ saved [1115394/1115394]



In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import numpy as np

In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(f"length in characters: {len(text)}")

length in characters: 1115394


In [ ]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
# mapping from char to int, and from into to char
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: takes a string, outputs a list of int
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: takes a list of int, outputs a string

print(encode("hi, there!"))
print(decode(encode("hi, there!")))

[46, 47, 6, 1, 58, 46, 43, 56, 43, 2]
hi, there!


In [ ]:
data = torch.tensor(encode(text), dtype=torch.long)

In [ ]:
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [ ]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8

In [ ]:
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is `{context}` the target is `{target}`")

when input is `tensor([18])` the target is `47`
when input is `tensor([18, 47])` the target is `56`
when input is `tensor([18, 47, 56])` the target is `57`
when input is `tensor([18, 47, 56, 57])` the target is `58`
when input is `tensor([18, 47, 56, 57, 58])` the target is `1`
when input is `tensor([18, 47, 56, 57, 58,  1])` the target is `15`
when input is `tensor([18, 47, 56, 57, 58,  1, 15])` the target is `47`
when input is `tensor([18, 47, 56, 57, 58,  1, 15, 47])` the target is `58`


In [ ]:
torch.manual_seed(42)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print(f'inputs: {xb.shape} \n{xb} \ntTargets: {yb.shape} \n{yb}')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target is {target}")

inputs: torch.Size([4, 8]) 
tensor([[57,  1, 46, 47, 57,  1, 50, 53],
        [ 1, 58, 46, 43, 56, 43,  1, 41],
        [17, 26, 15, 17, 10,  0, 32, 53],
        [57, 58,  6,  1, 61, 47, 58, 46]]) 
tTargets: torch.Size([4, 8]) 
tensor([[ 1, 46, 47, 57,  1, 50, 53, 60],
        [58, 46, 43, 56, 43,  1, 41, 39],
        [26, 15, 17, 10,  0, 32, 53,  1],
        [58,  6,  1, 61, 47, 58, 46,  0]])
when input is [57] the target is 1
when input is [57, 1] the target is 46
when input is [57, 1, 46] the target is 47
when input is [57, 1, 46, 47] the target is 57
when input is [57, 1, 46, 47, 57] the target is 1
when input is [57, 1, 46, 47, 57, 1] the target is 50
when input is [57, 1, 46, 47, 57, 1, 50] the target is 53
when input is [57, 1, 46, 47, 57, 1, 50, 53] the target is 60
when input is [1] the target is 58
when input is [1, 58] the target is 46
when input is [1, 58, 46] the target is 43
when input is [1, 58, 46, 43] the target is 56
when input is [1, 58, 46, 43, 56] the target is 43


In [ ]:
class BiGramLM(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.tk_emb_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.tk_emb_table(idx) # (Batch, Time, Channel)
        # logits = torch.permute(logits, (0, 2, 1))
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self(idx)
            # focus on last time step
            logits = logits[:, -1, :]
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to sequence
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [ ]:
model = BiGramLM(vocab_size)
logits, loss = model(xb, yb)
print(f"Logits Shape: {logits.shape} \nLoss: {loss} \nExpected Loss (-log(1/65)): {-np.log(1/65)}")

Logits Shape: torch.Size([32, 65]) 
Loss: 4.724130630493164 
Expected Loss (-log(1/65)): 4.174387269895637


In [ ]:
idx = torch.zeros((1,1), dtype=torch.long)
pred = decode(model.generate(idx, max_new_tokens=100)[0].tolist())
print(pred)


F$z
E?kFwu
'buM-,Y3fYNsA3xp.mpsAqaZ-RUldc;F
M$GfYCUCkFO-bJbz-R;O!slp.FNsJDV'jRzIMQ'EdRbqAoWTjrniaIIa


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [ ]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if steps % 1000 == 0 or steps == 10000:
        print(loss.item())

4.7295145988464355
3.8045549392700195
3.0847537517547607
2.7263264656066895
2.6540687084198
2.5964434146881104
2.6903610229492188
2.58370041847229
2.390960693359375
2.4909160137176514


In [ ]:
idx = torch.zeros((1,1), dtype=torch.long)
pred = decode(model.generate(idx, max_new_tokens=300)[0].tolist())
print(pred)


VAROLUSait; l ICotherer w war ha yevelise 'tWhowKEmurfor add hiull DULY Wod ongoz-berthHicirds wavente, m te kere isd, m INGea earusher Tistced.
Wharologo!


WhORDUCKETELK:
Tof?
GUFarastaldXfeshowoolicou myour d:
BERDou S:
GES por s bemue orinth'Top mofeawontill wintiryopiat fh o p--healy wecayse ho


# The Mathematical trick in self-attention

In [ ]:
# toy example
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b,t] = torch.mean(xprev, 0)

In [ ]:
x[0], xbow[0]

(tensor([[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [ ]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (/B, T, T) @ (B, T, C) -> (B, T, C)

In [ ]:
torch.allclose(xbow, xbow2)

False

In [ ]:
tril = torch.tril(torch.ones(T, T))
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [ ]:
wei = torch.zeros((T, T))
wei

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
wei = wei.masked_fill(tril == 0, float('-inf'))
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
wei = F.softmax(wei, dim=1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [ ]:
xbow3 = wei @ x

In [ ]:
torch.allclose(xbow, xbow3)

False

In [ ]:
torch.manual_seed(42)
a = torch.ones(3,3)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
print(f"a = \n{a} \nb = \n{b} \nc = \n{c} \n")

a = 
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]) 
b = 
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]]) 
c = 
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]]) 



In [ ]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [ ]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
print(f"a = \n{a} \nb = \n{b} \nc = \n{c} \n")

a = 
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]]) 
b = 
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]]) 
c = 
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]]) 



In [ ]:
torch.manual_seed(1337)
B,T,C = 4,8,32 # 4x8 arrangement of tokens, where each token is 32-dim
x = torch.randn(B,T,C)

# single head to perfomr self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) -> (B, T, T)

tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 16])

In [ ]:
wei[0]

tensor([[0.0248, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0052, 0.0091, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0521, 0.0135, 0.2482, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3171, 0.0214, 0.1642, 0.1188, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0412, 0.0487, 0.1046, 0.0742, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1060, 0.5347, 0.2059, 0.1030, 0.7402, 0.0192, 0.0000, 0.0000],
        [0.4298, 0.3409, 0.1769, 0.2027, 0.0480, 0.8472, 0.2329, 0.0000],
        [0.0238, 0.0316, 0.1002, 0.5013, 0.0117, 0.1336, 0.7671, 1.0000]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [ ]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5


In [ ]:
k.var()

tensor(1.0449)

In [ ]:
q.var()

tensor(1.0700)

In [ ]:
wei.var()

tensor(1.0918)

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1)


tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [ ]:
class LayerNorm1d(nn.Module):
    def __init__(self, dim, eps=1e-5):
        super().__init__()
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)

    def __call__(self, x):
        xmean = x.mean(dim=1, keepdim=True)
        xvar = x.var(dim=1, keepdim=True)
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        return self.out

    def parameters(self):
        return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100)
x = module(x)
x.shape

torch.Size([32, 100])

In [ ]:
x[0, :].mean(), x[:, 0].std()

(tensor(-9.5367e-09), tensor(0.8803))

In [ ]:
x[0, :].mean(), x[0, :].std()

(tensor(-9.5367e-09), tensor(1.0000))

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

block_size = 64
batch_size = 256
max_iters = 5000
eval_interval = 500
lr = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd_dim = 384
n_head = 6
n_layer = 6
dropout = 0.2

# download the file
# import urllib.request
# url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
# filename = 'input.txt'
# urllib.request.urlretrieve(url, filename)

with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

# mapping from char to int, and from into to char
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: takes a string, outputs a list of int
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: takes a list of int, outputs a string

data = torch.tensor(encode(text), dtype=torch.long)

n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            X, Y = X.to(device), Y.to(device)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

xb, yb = get_batch('train')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd_dim, head_size, bias=False)
        self.query = nn.Linear(n_embd_dim, head_size, bias=False)
        self.value = nn.Linear(n_embd_dim, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        # compute the attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        # weighted aggregation of values
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd_dim, n_embd_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([head(x) for head in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd_dim, 4 * n_embd_dim),
            nn.ReLU(),
            nn.Linear(4 * n_embd_dim, n_embd_dim), # projection layer
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """Transformer Block."""
    def __init__(self, n_embd_dim, n_heads):
        super().__init__()
        head_size = n_embd_dim // n_heads
        self.sa = MultiHeadAttention(n_heads, head_size) # communication
        self.ffwd = FeedForward(n_embd_dim) # computation
        self.ln1 = nn.LayerNorm(n_embd_dim)
        self.ln2 = nn.LayerNorm(n_embd_dim)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class BiGramLM(nn.Module):
    def __init__(self):
        super().__init__()
        self.tk_emb_table = nn.Embedding(vocab_size, n_embd_dim)
        self.position_emb_table = nn.Embedding(block_size, n_embd_dim)
        self.blocks = nn.Sequential(*[Block(n_embd_dim, n_heads=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd_dim)
        self.lm_head = nn.Linear(n_embd_dim, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tk_embd = self.tk_emb_table(idx) # (Batch, Time, Channel)
        pos_embd = self.position_emb_table(torch.arange(T, device=idx.device)) # (Time, Channel)
        x = tk_embd + pos_embd
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        # logits = torch.permute(logits, (0, 2, 1))
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # crop the idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get predictions
            logits, _ = self(idx_cond)
            # focus on last time step
            logits = logits[:, -1, :]
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to sequence
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

model = BiGramLM().to(device)
print(sum(p.numel() for p in model.parameters()) / 1e6, 'M Parameters')

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

for it in range(max_iters):
    model.train()
    xb, yb = get_batch('train')
    xb, yb = xb.to(device), yb.to(device)
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if it % eval_interval == 0 or it == max_iters - 1:
        losses = estimate_loss()
        print(f'Iter {it:4d}, Train loss: {losses["train"]:.2f}, Val loss: {losses["val"]:.2f}')

context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(model.generate(context, 500)[0].cpu().numpy()))

Iter    0, Train loss: 3.63, Val loss: 3.65
Iter  500, Train loss: 1.65, Val loss: 1.81
Iter 1000, Train loss: 1.43, Val loss: 1.63
Iter 1500, Train loss: 1.34, Val loss: 1.56
Iter 2000, Train loss: 1.29, Val loss: 1.54
Iter 2500, Train loss: 1.24, Val loss: 1.52
Iter 3000, Train loss: 1.20, Val loss: 1.52
Iter 3500, Train loss: 1.17, Val loss: 1.51
Iter 4000, Train loss: 1.14, Val loss: 1.51
Iter 4500, Train loss: 1.11, Val loss: 1.52
Iter 4999, Train loss: 1.08, Val loss: 1.52

And by the Messenger's honour.

Second Citizen:
For every tongue
were sensed in our deputy,
And show'd your most such and feasting eyes' putting
And justice that proclamations may jewly,
But villainly


In [ ]:
print(decode(model.generate(context, 2500)[0].cpu().numpy()))


Gaunt:
What wars he shall. His rest, he means three be he'ld.

HENRY BOLINGBROKE:
Joy lord, you tell the trumpet unite,
Amity here, as I mean o' the fancy, in
arguession's service, or looks nothing on servery, corn'd
The maids of night doings doubly upon of God;
And sentench their orwards are off allowed?
For layful bague I draw his free-a.
Thush Royal is held our much;
But where he shall now see, this country's death?

Nurse:
These speech is something?

LADY CAPULET:
A honour, that have they can leave a weak unto the touchesy.

BENVOLIO:
For that thou art noble, perusequito, farewell.

CLARENCE:
Sir, I am o, or too the face of naturing war
Is not to be preselved but his new.

KING HENRY VI:
O JOhince, let my son, I should wrought.

Gaoler:
Sirrah God, will I look your hand unworston.
What moverwhy, are you may do?

POMPEY:
Your hands, your dispersions married with took-night:
Too horroy good succession, right hath here to compell it
On the stars which is the corse.

Clown:
Ay, coward